In [ ]:
#####設定讀取資料的年份與時段#####

year = 104    #110年或104年
rush_hour = '晨峰'    #'晨峰'或'昏峰'

inputLocation = f"{year}_現況_{rush_hour}/"

################################

import pandas as pd
import numpy as np

# 讀取 Excel 檔案，處理空值
df = pd.read_excel("TRTS-4S屏柵線.xlsx", sheet_name=None)
sheet1 = df["節點編號"]

# 處理 sheet1 中的空值
sheet1 = sheet1.fillna(0)  # 將空值填充為 0
sheet1 = sheet1.replace('', 0)  # 將空字串替換為 0

if rush_hour == '晨峰':
    real_flow = df["晨峰實際流量"]
elif rush_hour == '昏峰':
    real_flow = df["昏峰實際流量"]

# 處理 real_flow 中的空值
real_flow = real_flow.fillna(0)
real_flow = real_flow.replace('', 0)

print(f'——————————————————————校估{year}年{rush_hour}指派流量——————————————————————')

real_flow = real_flow.set_index("group")

ue_results = pd.read_csv(inputLocation + f"{year}_現況_{rush_hour}_UE_results.dat", sep="\t")
ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

ue_results["link_name"] = ue_results.apply(lambda row: f"{int(row['A'])}_{int(row['B'])}", axis=1)

# 建立 link_name 時處理可能的空值
def safe_link_name(a, b):
    """安全地建立 link_name，處理空值"""
    try:
        if pd.isna(a) or pd.isna(b) or a == 0 or b == 0:
            return "0_0"  # 空值或 0 的情況
        return f"{int(a)}_{int(b)}"
    except (ValueError, TypeError):
        return "0_0"

sheet1["link_name1"] = sheet1.apply(lambda row: safe_link_name(row['A1'], row['B1']), axis=1)
sheet1["link_name2"] = sheet1.apply(lambda row: safe_link_name(row['A2'], row['B2']), axis=1)

# 合併資料時處理空值
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], 
                      left_on="link_name1", right_on="link_name", how="left")
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], 
                      left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))

# 處理合併後的空值
sheet1['UE_flow_1'] = sheet1['UE_flow_1'].fillna(0)
sheet1['UE_flow_2'] = sheet1['UE_flow_2'].fillna(0)

# 保留需要的欄位
sheet1 = sheet1[["group","link", "UE_flow_1", "UE_flow_2"]]

# 處理 group 可能為空值的情況
sheet1 = sheet1.dropna(subset=['group'])  # 移除 group 為空值的行

group_names = sheet1['group'].unique()
groups = {i: {'UE_flow_1': 0, 'UE_flow_2': 0} for i in group_names}

for index, row in sheet1.iterrows():
    group = row['group']
    # 確保流量值不是 NaN
    ue_flow_1 = row['UE_flow_1'] if not pd.isna(row['UE_flow_1']) else 0
    ue_flow_2 = row['UE_flow_2'] if not pd.isna(row['UE_flow_2']) else 0
    
    groups[group]['UE_flow_1'] += ue_flow_1
    groups[group]['UE_flow_2'] += ue_flow_2

# UE_flow_1 代表(往南/東)流量， UE_flow_2 代表(往北/西)流量
for group, flows in groups.items():
    try:
        # 檢查 real_flow 中是否有該 group 的資料
        if group not in real_flow.index:
            print(f"【{group}】 資料不完整，跳過計算")
            continue
            
        # 安全地取得實際流量數據
        name = real_flow.loc[group, 'name'] if 'name' in real_flow.columns else f"Group {group}"
        south_east = real_flow.loc[group, '南/東'] if '南/東' in real_flow.columns and not pd.isna(real_flow.loc[group, '南/東']) else 1
        north_west = real_flow.loc[group, '北/西'] if '北/西' in real_flow.columns and not pd.isna(real_flow.loc[group, '北/西']) else 1
        
        # 計算差異百分比，避免除以零
        model_diff_se = (flows['UE_flow_1'] - south_east) / south_east * 100 if south_east != 0 else 0
        model_diff_nw = (flows['UE_flow_2'] - north_west) / north_west * 100 if north_west != 0 else 0
        
        # CUBE 差異計算（如果有 CUBE 資料）
        cube_se = real_flow.loc[group, 'CUBE南/東'] if 'CUBE南/東' in real_flow.columns and not pd.isna(real_flow.loc[group, 'CUBE南/東']) else south_east
        cube_nw = real_flow.loc[group, 'CUBE北/西'] if 'CUBE北/西' in real_flow.columns and not pd.isna(real_flow.loc[group, 'CUBE北/西']) else north_west
        
        cube_diff_se = (cube_se - south_east) / south_east * 100 if south_east != 0 else 0
        cube_diff_nw = (cube_nw - north_west) / north_west * 100 if north_west != 0 else 0
        
        print(f"""【{group}】 {name}
    UE flow 往南/東 = {round(flows['UE_flow_1'])}， model diff = {model_diff_se:.2f}%， CUBE diff = {cube_diff_se:.2f}%
    UE flow 往北/西 = {round(flows['UE_flow_2'])}， model diff = {model_diff_nw:.2f}%， CUBE diff = {cube_diff_nw:.2f}%
    """)
        
    except Exception as e:
        print(f"【{group}】 處理時發生錯誤: {e}")
        continue

In [ ]:
# sheet1 sorted by group and link
sheet1.sort_values(by=['group', 'link'], inplace=True)
pd.set_option('display.max_rows', None)
sheet1[["group", "link", "UE_flow_1", "UE_flow_2"]]
#show all the results in a table